In [11]:
!git clone https://github.com/KodeWil/final_project_AI_csv_Data.git

fatal: destination path 'final_project_AI_csv_Data' already exists and is not an empty directory.


In [0]:
#Dependencies
# %reset
from sklearn.svm import SVC
import pandas as pd 
import glob 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import copy
from sklearn.metrics import accuracy_score
import random as rn
from sklearn import preprocessing

In [0]:
def addData(files: str) -> list:
    allCsv = []
    for file in files:
      temp = open(file)
      tempDf = pd.read_csv(file)
      allCsv.append(tempDf)
    return allCsv



In [0]:
def printAll(df):
  pd.set_option('display.max_rows', None)
  print(df)

In [15]:
frames = []

#Create a list ('frames') with four nested lists, one per sensor. 
#Each nested list have all the samples (DF) for that sensor.
%cd /content/final_project_AI_csv_Data/phone/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/phone/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/accel
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
%cd /content/final_project_AI_csv_Data/watch/gyro
files = glob.glob('*.csv')
files.sort()
frames.append(addData(files))
print(len(frames))
# 0 in frames = /phone/accel
# 1 in frames = /phone/gyro
# 2 in frames = /watch/accel
# 3 in frames = /watch/gyro

/content/final_project_AI_csv_Data/phone/accel
/content/final_project_AI_csv_Data/phone/gyro
/content/final_project_AI_csv_Data/watch/accel
/content/final_project_AI_csv_Data/watch/gyro
4


In [0]:
def cutSamples(frames1, frames2):
  for i in range (0,50):
    samples1 = frames1[i].shape[0]
    samples2 = frames2[i].shape[0]
    if(samples1 > samples2):
      toDrop = list(range(samples2, samples1))
      frames1[i].drop(frames1[i].index[[toDrop]], inplace = True)
    elif(samples2 > samples1):
      toDrop = list(range(samples1, samples2))
      frames2[i].drop(frames2[i].index[[toDrop]], inplace = True) 


In [0]:
def SVM(X, Xte, y, yte)->tuple:
  kernelTypes = ["rbf", "linear" ,"poly", "sigmoid"]
  gammaTypes = ["auto", "scale"]
  class_weights = [True, False]
  bestScore = 0
  for kernelType in kernelTypes:
    for gammaType in gammaTypes: 
      for weight in class_weights:
          if(weight):
            clf = SVC(kernel= kernelType, degree= 2, 
                      gamma= gammaType,class_weight= 'balanced')
            parameters = {"kernel": kernelType,  "gamma": gammaType, "weight": "balanced"}
          else:
            clf = SVC(kernel= kernelType, degree= 3, 
                      gamma= gammaType)
            parameters = {"kernel": kernelType,  "gamma": gammaType, "weight": "None"}
          # print(parameters)
          clf.fit(X,y)  
          tempScore = clf.score(Xte, yte)
          if(tempScore > bestScore):
            bestScore = tempScore
            svmModel = (parameters, bestScore)
  return svmModel

In [0]:
def svmParticular(X, Xte, y, yte, kernelType, gammaType, weight):
  if(weight):
      clf = SVC(kernel= kernelType, degree= 2, 
                gamma= gammaType,class_weight= 'balanced')
      parameters = {"kernel": kernelType,  "gamma": gammaType, "weight": "balanced"}
  else:
      clf = SVC(kernel= kernelType, degree= 3, 
                gamma= gammaType)
      parameters = {"kernel": kernelType,  "gamma": gammaType, "weight": "None"}
  
  clf.fit(X,y)  
  score = clf.score(Xte, yte)
  print(score)

In [0]:
#Split into train and test data, encode "ACTIVITY" column into a numerical column
def dataTransformation(frame, columns: bool, kernelType, gammaType, weight, final):
  ohe = OneHotEncoder()
  sc = StandardScaler()
  le = preprocessing.LabelEncoder()
  if(columns):
    y = frame["ACTIVITY"]
    X = frame.drop(columns = ["ACTIVITY", "class"])
    X = X.values
    X = sc.fit_transform(X)
    y = y.values
    le.fit(y)
    y = le.transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    if(final):
      info = SVM(X_train, X_test, y_train, y_test)
      return(info)
    else:
      svmParticular(X_train, X_test, y_train, y_test, kernelType, gammaType, weight)

  else:
    y = frame[0]
    y = y.values
    le.fit(y)
    y = le.transform(y)
    X = frame.drop(columns = [0,91,93,182])
    X = X.values
    X = sc.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    info = svmParticular(X_train, X_test, y_train, y_test, kernelType, gammaType, weight)
    return(info)







In [20]:
#Apply the models to some individuals samples 
scores = []
names = ["Phone Accel", "Phone Gyro" , "Watch Accel", "Watch Gyro"]
numSamples = 10
sensorScore = []
print("For a single sample")
for i, frame in enumerate(frames):
  print("-------------------------------------------")
  print(names[i])
  print("")
  avg = 0
  for i in range(0,numSamples):
    tempVal = rn.randint(0,49)
    sensorScore.append(dataTransformation(frame[tempVal], True,"","","",True))
    avg += sensorScore[i][1]
  avg = round(avg/numSamples, 3)
  best = max(sensorScore, key = lambda x:x[1])
  print("Parameters for best model: " + str(best[0]) + " Average score:" + str(avg))
  print("")
  sensorScore.clear()
print('---------------------------------------------------------------------------------------------------------------------------------------')
print("")
print("")


# Concatenate all phone Dataframes in a single One
sensors = []
phoneAccel = pd.concat(frames[0])
sensors.append(phoneAccel)
phoneGyro = pd.concat(frames[1])
sensors.append(phoneGyro)
watchAccel = pd.concat(frames[2])
sensors.append(watchAccel)
watchGyro = pd.concat(frames[3])
sensors.append(watchGyro)

#Scores for single sensors


print("")
print("Phone Accelerometer sensor accuracy")
print("Score: ")
dataTransformation(phoneAccel, True,"rbf", "scale", False,False)
print("")

print("")
print("Watch Accelerometer sensor accuracy")
print("Score: ")
dataTransformation(watchAccel, True,"rbf", "auto", True, False)
print("")

print("")
print("Phone Gyroscope sensor accuracy")
print("Score: " )
dataTransformation(phoneGyro, True,"rbf", "scale", False,False)
print("")

print("")
print("Watch Gyroscope sensor accuracy")
print("Score: ")
dataTransformation(watchGyro, True,"rbf", "scale", True,False)
print("")



#Cut samples to match with the number of samples in the other sensor (accel and gyro)
cutSamples(frames[0], frames[1]) #Phone sensors
cutSamples(frames[2], frames[3]) #Watch sensors

#Concat all individual samples for a single sensor in one data frame. 
phoneAccel = pd.concat(frames[0], ignore_index=True)
phoneGyro = pd.concat(frames[1], ignore_index=True)
watchAccel = pd.concat(frames[2], ignore_index=True)
watchGyro = pd.concat(frames[3], ignore_index=True)

#Create a new data frame with both sensors for both devices.
phone = pd.concat([phoneAccel, phoneGyro], ignore_index=True, axis = 1)
phoneIndex = phone[phone[0] != phone[93]].index
phone.drop(phoneIndex, inplace=True)
watch = pd.concat([watchAccel, watchGyro], ignore_index=True, axis = 1)
watchIndex = watch[watch[0] != watch[93]].index
watch.drop(watchIndex, inplace=True)

#Score for merged devices sensors
print("")
print("Phone merged sensors accuracy")
print("Score: ")
dataTransformation(phone, False,"rbf", "scale", True, False)
print("")

print("")
print("Watch merged sensors accuracy")
print("Score: ")
dataTransformation(watch, False, "rbf", "scale", False, False)
print("")



For a single sample
-------------------------------------------
Phone Accel

Parameters for best model: {'kernel': 'rbf', 'gamma': 'auto', 'weight': 'balanced'} Average score:0.887

-------------------------------------------
Phone Gyro

Parameters for best model: {'kernel': 'rbf', 'gamma': 'auto', 'weight': 'balanced'} Average score:0.756

-------------------------------------------
Watch Accel

Parameters for best model: {'kernel': 'linear', 'gamma': 'auto', 'weight': 'balanced'} Average score:0.942

-------------------------------------------
Watch Gyro

Parameters for best model: {'kernel': 'rbf', 'gamma': 'auto', 'weight': 'balanced'} Average score:0.851

---------------------------------------------------------------------------------------------------------------------------------------



Phone Accelerometer sensor accuracy
Score: 
0.6838569880823402


Watch Accelerometer sensor accuracy
Score: 
0.8114191600329399


Phone Gyroscope sensor accuracy
Score: 
0.4197280879375181


W

/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/base.py:3941: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


0.744265495363592


Watch merged sensors accuracy
Score: 
0.8552500828088772

